In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import cv2
print('OpenCV version used:', cv2.__version__)

from skimage import io
from matplotlib import pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10,10)

# Affine Transformation: Task 1.2
- Given 3 pairs of points ﬁnd afﬁne transformation and apply it to an image.
- Compare with OpenCV function

In [ ]:
def url_to_image(url):
    print("downloading %s" % (url))
    return io.imread(url)

def imshow(img, cmap='gray', sub=None, title=None, ax='off'):
    if sub is not None:
        plt.subplot(*sub)
    if title is None:
        title = img.shape
    plt.title(title)
    plt.imshow(img, cmap=cmap)
    plt.grid(False)
    plt.axis(ax)
    if sub is None:
        plt.show()

url = "https://raw.githubusercontent.com/ucuapps/computer-vision-course/master/module4-geometry/practice2_homography/img/view.jpg"
img = url_to_image(url)[:500,:,:]
imshow(img)

In [ ]:
# Synthesize point correspondences {x1 <-> x2}

(h, w) = img.shape[:2]

x1 = np.float32([[0, 0],
                 [w, 0],
                 [0, h]])

x2 = np.float32([[50 ,0],
                 [w, 50],
                 [0, h - 50]])

In [ ]:
# Get affine transformation with OpenCV

A = cv2.getAffineTransform(x1, x2)
print(A)

dst = cv2.warpAffine(img, A, (w,h))
imshow(img, sub=(1,2,1))
imshow(dst, sub=(1,2,2))
plt.show()

In [ ]:
def getAffineTransform(x1, x2):

#def getAffineTransform(original_points, transformed_points):
    p = []
    for x,y in x1:
        p.append((x,y,1))
    return np.linalg.solve(p, x2).T

In [ ]:
A2 = getAffineTransform(x1, x2)
print(A2)

assert(np.linalg.norm(A2 - A, 'fro') < 1e-12)

dst = cv2.warpAffine(img, A2, (w,h))
imshow(img, sub=(1,2,1))
imshow(dst, sub=(1,2,2))
plt.show()

# Projective Transformation: Task 2.2
- Given 4 pairs of points find projective transformation and apply it to an image. 
- Compare with OpenCV function

In [ ]:
url = "https://raw.githubusercontent.com/ucuapps/computer-vision-course/master/module4-geometry/practice2_homography/img/cvbook.jpg"
img = url_to_image(url)
imshow(img)

In [ ]:
# Get point correspondences {x1 <-> x2}

x1 = np.float32([[241, 6],
                 [726, 29],
                 [6, 668],
                 [627, 837]])

x2 = np.float32([[0, 0],
                 [300, 0],
                 [0, 400],
                 [300, 400]])

In [ ]:
# Get homography with OpenCV

H = cv2.getPerspectiveTransform(x1, x2)
H /= H[2,2]
print(H)

dst = cv2.warpPerspective(img, H, (300,400))
imshow(img, sub=(1,2,1))
imshow(dst, sub=(1,2,2))
plt.show()

In [ ]:
def get_coef(a, b, n):
    res = []
    b = [b[0], b[1], 1]
    dim = 3
    for i in range(dim):
        curr = [0] * dim * 4
        curr[i] = a[0]
        curr[dim + i] = a[1]
        curr[2*dim + i] = 1 if i != 2 else 0
        
        curr[3*dim + n - 1] = -b[i]
        res.append(curr)
        
    return res

In [ ]:
#def getPerspectiveTransform(x1,x2):
def getPerspectiveTransform(pts1, pts2):
    A = []
    plen = len(pts1)
    for i in range(plen):
        A += get_coef(pts1[i], pts2[i], i)
        
    B = [0, 0, -1] * plen
    C = np.linalg.solve(A, B)
    res = np.ones(9)
    res[:8] = C.flatten()[:8]
    return res.reshape(3,-1).T

In [ ]:
H2 = getPerspectiveTransform(x1, x2)
H2 /= H2[2,2]
print(H2)
assert(np.linalg.norm(H2 - H, 'fro') < 1e-12)

dst = cv2.warpPerspective(img, H2, (300, 400))
imshow(img, sub=(1,2,1))
imshow(dst, sub=(1,2,2))
plt.show()

# Correspondence and Matching: Task 3.1

In [ ]:
url1 = "https://raw.githubusercontent.com/ucuapps/computer-vision-course/master/module4-geometry/practice2_homography/img/bookflat.jpg"
url2 = "https://raw.githubusercontent.com/ucuapps/computer-vision-course/master/module4-geometry/practice2_homography/img/cvbook.jpg"
img1 = cv2.resize(url_to_image(url1), (0,0), fx=0.5, fy=0.5)
img2 = cv2.resize(url_to_image(url2), (0,0), fx=0.5, fy=0.5)
imshow(img1, sub=(1,2,1))
imshow(img2, sub=(1,2,2))
plt.show()

In [ ]:
# 

orb = cv2.ORB_create()
# find the keypoints and descriptors with ORB
# because SIFT and SUFT are patented (Anticipated expiration 2020-03-06)
kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)

# create BFMatcher object (Brute-force descriptor matcher)
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Match descriptors.
matches = bf.match(des1,des2)

# Sort them in the order of their distance.
matches = sorted(matches, key = lambda x:x.distance)

# Draw first 30 matches.
img3 = cv2.drawMatches(img1, kp1, img2, kp2, matches[:30], None, flags=2)

imshow(img3)

In [ ]:
# Get homography with OpenCV

if len(matches)>4:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in matches ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in matches ]).reshape(-1,1,2)

    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()

    h,w = img1.shape[:2]
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts, H)

    img4 = cv2.polylines(img2, [np.int32(dst)],True,255,3, cv2.LINE_AA)
    img4_rectified = cv2.warpPerspective(img4, np.linalg.inv(H), (150, 200))
    imshow(img4, sub=(1,2,1))
    imshow(img4_rectified, sub=(1,2,2))
    plt.show()

else:
    print("Not enough matches are found - %d/%d" % (len(matches), MIN_MATCH_COUNT))
    matchesMask = None

In [ ]:
def findHomography(x1, x2):
    ########### please implement me ###########
    pass
    ###########################################

In [ ]:
if len(matches)>4:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in matches ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in matches ]).reshape(-1,1,2)

    H, mask = findHomography(src_pts, dst_pts)
    matchesMask = mask.ravel().tolist()

    h,w = img1.shape[:2]
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts, H)

    img4 = cv2.polylines(img2, [np.int32(dst)],True,255,3, cv2.LINE_AA)
    img4_rectified = cv2.warpPerspective(img4, np.linalg.inv(H), (150, 200))
    imshow(img4, sub=(1,2,1))
    imshow(img4_rectified, sub=(1,2,2))
    plt.show()

else:
    print("Not enough matches are found - %d/%d" % (len(matches), MIN_MATCH_COUNT))
    matchesMask = None